In [36]:
import numpy as np
from scipy import signal

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import scipy.io
import sklearn.metrics as skmetrics
import time
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# load numpy array from csv file
from numpy import loadtxt

In [20]:
"""
Function for initializing the graphic Equalizer

    Parameters
    ----------
    G_db : ndarray 31x1
        Command gains in dB
        
    Returns
    -------
    numsopt : ndarray
        numerator parts of the 31 filters
    densopt : ndarray
        denominator parts of the 31 filters
    fs : float
        sample frequency
    fc2 : ndarray
        center frequencies and additional design points between them
    G_db2 : ndarray
        interpolate target gains linearly b/w command gains
        
    Notes
    -----
    Python reference to Liski, J.; Välimäki, V. The quest for the best graphic equalizer. In Proceedings of the International Conference
    on Digital Audio Effects (DAFx-17), Edinburgh, UK, 5–9 September 2017; pp. 95–102

"""


def initGEQ(G_db):
    fs = 44.1e3
    fc1 = np.array([19.69,24.80,31.25,39.37,49.61,62.50,78.75,99.21,125.0,157.5,198.4,
    250.0,315.0,396.9,500.0,630.0,793.7,1000,1260,1587,2000,2520,3175,4000,
    5040,6350,8000,10080,12700,16000,20160])
    fc2 = np.zeros(61)
    fc2[::2] = fc1
    fc2[1::2] = np.sqrt(fc1[0:len(fc1)-1:1] * fc1[1::1])
    wg = 2*pi*fc1/fs
    wc = 2*pi*fc2/fs
    c= 0.38
    bw = np.array((2 ** (1/3) - 2 ** (-1/3)) * wg)
    bw_adjust = 2*pi/fs
    bw[::] =[9.178*bw_adjust, 11.56*bw_adjust, 14.57*bw_adjust, 18.36*bw_adjust, 23.13*bw_adjust, 29.14*bw_adjust, 36.71*bw_adjust, 46.25*bw_adjust, 58.28*bw_adjust, 73.43*bw_adjust, 
    92.51*bw_adjust, 116.6*bw_adjust, 146.9*bw_adjust, 185.0*bw_adjust, 233.1*bw_adjust, 293.7*bw_adjust,370*bw_adjust, 466.2*bw_adjust, 587.4*bw_adjust, 740.1*bw_adjust, 932.4*bw_adjust, 1175*bw_adjust, 1480*bw_adjust, 1865*bw_adjust, 2350*bw_adjust, 2846*bw_adjust, 3502*bw_adjust, 4253*bw_adjust, 5038*bw_adjust, 5689*bw_adjust, 5570*bw_adjust]
    leak = interactionMatrix(10**(17/20)*np.ones(31),c,wg,wc,bw)
    

    G_db2 = np.zeros([61,1])
    G_db2[::2] = G_db
    G_db2[1::2] = (G_db[:len(G_db)-1:1]+G_db[1::1])/2
    
    Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
    Gopt = 10**(Gopt_db/20)
    
    leak2 = interactionMatrix(Gopt,c,wg,wc,bw)
    G2opt_db = np.linalg.lstsq(leak2.conj().T, G_db2)[0] #filter gains
    G2opt = 10 **(G2opt_db/20)
    G2wopt_db = c * G2opt_db
    G2wopt = 10 **(G2wopt_db/20)
    
    numsopt = np.zeros((3,31))
    densopt = np.zeros((3,31))
    
    for k in range(31):
        [num,den] = pareq(G2opt[k],G2wopt[k],wg[k],bw[k])
        numsopt[:,k] = num
        densopt[:,k] = den
   
    return numsopt,densopt,fs,fc2,G_db2,G2opt_db,fc1,bw

In [21]:
"""
Second-order parametric equalizing filter desigh with adjustable bandwidth gain

    Parameters
    ----------
    G : float64
        peak gain (linear)
    GB : float64
        bandwidth gain (linear)
    w0: float64
        center frequency (rads/sample)
    B : float64
        bandwidth (rads/sample)
    
    Returns
    -------
    num : ndarray
        numerator coefficients [b0,b1,b2]
    den : ndarray
        denominator coefficients [1,a1,a2]
        
    Notes
    -----
    Python reference to Liski, J.; Välimäki, V. The quest for the best graphic equalizer. In Proceedings of the International Conference
    on Digital Audio Effects (DAFx-17), Edinburgh, UK, 5–9 September 2017; pp. 95–102
    
"""
def pareq(G, GB, w0, B):
    if G == 1:
        beta = tan(B/2.)
    else: 
        beta = np.sqrt(abs(GB**2-1)/abs(G**2-GB**2))*tan(B/2)

    num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
    den = np.array([1, -2*cos(w0)/(1+beta), (1-beta)/(1+beta)])
    
    return num, den

In [22]:
"""
Compute the interaction matrix of a cascade GEQ

    Parameters
    ----------
    G : ndarray
        linear gain at which the leakage is determined    
    c : float
        gain factor at bandwidth (0.5 refers to db(G)/2)
    wg : ndarray
        command frequencies i.e. filter center frequencies (in rad/sample)
    wc : ndarray
        design frequencies (rad/sample) at which leakage is computed
    bw : ndarray 
        bandwidth of filters in radians
        
    Returns
    -------
    leak : ndarray
        N by M matrix showing how the magnitude responses of the band filters leak to the design frequencies.
        N is determined from the length of the array wc (number of design frequencies) whereas M is 
        determined from the length of wg (number of filters)
        
    Notes
    -----
    Python reference to Liski, J.; Välimäki, V. The quest for the best graphic equalizer. In Proceedings of the International Conference
    on Digital Audio Effects (DAFx-17), Edinburgh, UK, 5–9 September 2017; pp. 95–102
"""

def interactionMatrix(G,c,wg,wc,bw):

    M = len(wg)
    N = len(wc)
    leak = np.zeros([M,N]) 
    Gdb = 20 * np.log10(G)
    Gdbw = c * Gdb
    Gw = 10 ** (Gdbw/20)
    
    for m in range(M):
        [num, den] = pareq(G[m],Gw[m],wg[m],bw[m])
        w,h = signal.freqz(num, den, wc)
        Gain = 20*np.log10(np.abs(h))/Gdb[m]
        leak[m,:] = np.abs(Gain)
        
    return leak

In [23]:
def plotAbsoluteError(numsopt,densopt,fs,G_db,fc1):
    N_freq = 31
    w = fc1
    H_opt = np.ones((N_freq,31), dtype=complex)
    H_opt_tot = np.ones((N_freq,1), dtype=complex)
    
    for k in range(31):
        w, h = signal.freqz(numsopt[:,k], densopt[:,k],worN=w,fs=fs)
        H_opt[:,k]= h
        H_opt_tot = H_opt[:,[k]]  * H_opt_tot
    

    e1 = 20*np.log10(np.abs(H_opt_tot)).T
    error = np.abs(e1[0].reshape((31,1))-G_db.reshape((31,1)))
    
    plt.semilogx(w,error)
    plt.ylabel("Fehler in dB")
    plt.xlabel("Frequenz in Hz")
    #plt.title("Absolute Error")
    #plt.xticks([10, 30, 100, 1000, 3000, 10000])
    #plt.yticks(np.arange(-20,20,5))
    plt.grid(which="both", linestyle="--", color="grey")
    plt.show()
    
    
    

In [24]:
def ErrorFunction(numsopt,densopt,fs,G_db,fc1):
    N_freq = 31
    w = fc1
    H_opt = np.ones((N_freq,31), dtype=complex)
    H_opt_tot = np.ones((N_freq,1), dtype=complex)
    
    for k in range(31):
        w, h = signal.freqz(numsopt[:,k], densopt[:,k],worN=w,fs=fs)
        H_opt[:,k]= h
        H_opt_tot = H_opt[:,[k]]  * H_opt_tot
    
    e1 = 20*np.log10(np.abs(H_opt_tot)).T

    error = np.abs(e1.reshape((31,1))-G_db.reshape((31,1)))
    error_MAE = mean(error)     # = MAE
    MSE = skmetrics.mean_squared_error(e1[0], G_db)
    RMSE = np.sqrt(MSE)
    MAE = skmetrics.mean_absolute_error(e1[0], G_db)
    
    return MSE, RMSE, MAE

In [25]:
"""
Evaluate and plot

    Parameters
    ----------
    numsopt : ndarray
        numerator coefficients for each filter
    densopt : ndarray
        denominator coefficients for each filter
    fs : float
        sample frequency
    fc2 : ndarray
        center frequencies and additional design points between them
    G_db2 : ndarray
        interpolate target gains linearly b/w command gains
        
    Returns
    -------
    
    Notes
    -----
    Python reference to Liski, J.; Välimäki, V. The quest for the best graphic equalizer. In Proceedings of the International Conference
    on Digital Audio Effects (DAFx-17), Edinburgh, UK, 5–9 September 2017; pp. 95–102
    
"""

def plot(numsopt,densopt,fs,fc2,G_db2,G2opt_db,fc1):
    N_freq = 2 **12
    w = np.logspace(np.log10(9),np.log10(22050), N_freq)
    H_opt = np.ones((N_freq,31), dtype=complex)
    H_opt_tot = np.ones((N_freq,1), dtype=complex)
    
    for k in range(31):
        w, h = signal.freqz(numsopt[:,k], densopt[:,k],worN=w,fs=fs)
        H_opt[:,k]= h
        H_opt_tot = H_opt[:,[k]]  * H_opt_tot
    
    
    
    fig = figure(200)
    plt.semilogx(w,20*np.log10(np.abs(H_opt_tot)))
    #plt.semilogx(w,20*np.log10(np.abs(H_opt)))
    plt.plot(fc2,G_db2, "ro", markersize=5, markerfacecolor="none")
    plt.plot(fc1,G2opt_db, "ro", markersize=5, markerfacecolor="none", marker="x")
    plt.ylabel("Pegel in dB")
    plt.xlabel("Frequenz in Hz")
    #plt.title("Frequency response")
    plt.xticks([10, 30, 100, 1000, 3000, 10000])
    plt.yticks(np.arange(-15,20,5))
    plt.grid(which="both", linestyle="--", color="grey")
    filtergain = mlines.Line2D([], [], linestyle='None',
                          markersize=8, markerfacecolor="none", marker="x", markeredgecolor="r", label="Filter Gains")
    targetgain = mlines.Line2D([], [], linestyle='None',
                          markersize=8, markerfacecolor="none", marker="o", markeredgecolor="r", label="Target Gains")
    plt.legend(handles=[filtergain,targetgain])
    #plt.show()
    
    
    #fig.savefig("Figures/Opt_alter12dB.pdf")
    
    return fig

In [26]:
def plotGdbError(G_db,fs,fc2,fc1,bw):
   
    G_db2 = np.zeros([61,1])
    G_db2[::2] = G_db
    G_db2[1::2] = (G_db[:len(G_db)-1:1]+G_db[1::1])/2
    wg = 2*pi*fc1/fs
    c = 0.38
    # to get bad frequency response
    numsopt = np.zeros((3,31))
    densopt = np.zeros((3,31))
    G = 10 **(G_db/20)
    Gw_db = c * G_db
    Gw = 10 **(Gw_db/20)
    
    for k in range(31):
        [num,den] = pareq(G[k],Gw[k],wg[k],bw[k])
        numsopt[:,k] = num
        densopt[:,k] = den

    N_freq = 2 **12
    w = np.logspace(np.log10(9),np.log10(22050), N_freq)
    H_opt = np.ones((N_freq,31), dtype=complex)
    H_opt_tot = np.ones((N_freq,1), dtype=complex)
    
    for k in range(31):
        w, h = signal.freqz(numsopt[:,k], densopt[:,k],worN=w,fs=fs)
        H_opt[:,k]= h
        H_opt_tot = H_opt[:,[k]]  * H_opt_tot
    
    
    plt.semilogx(w,20*np.log10(np.abs(H_opt_tot)))
    #plt.semilogx(w,20*np.log10(np.abs(H_opt)))
    #plt.plot(fc2,G_db2, "ro", markersize=6, markerfacecolor="none")
    plt.plot(fc1,G_db, "ro", markersize=6, markerfacecolor="none", marker="x", markeredgecolor="r", label="Command Gains")
    plt.ylabel("Pegel in dB")
    plt.xlabel("Frequenz in Hz")
    #plt.title("Frequency Response")
    plt.xticks([10, 30, 100, 1000, 3000, 10000])
    plt.yticks(np.arange(-15,15,5))
    plt.grid(which="both", linestyle="--", color="grey")
    filtergain = mlines.Line2D([], [], linestyle='None',
                          markersize=8, markerfacecolor="none", marker="x", markeredgecolor="r", label="Filter Gains")
    plt.legend(handles=[filtergain])
    
    #plt.savefig("Figures/NoOpt_alter12dB.pdf")
    plt.show()
    
    return 

In [27]:
def plotPredictions(filtergainsPrediction,G_db,fs,fc2,fc1,bw):
    
    G_db2 = np.zeros([61,1])
    G_db2[::2] = G_db
    G_db2[1::2] = (G_db[:len(G_db)-1:1]+G_db[1::1])/2
    wg = 2*pi*fc1/fs
    c = 0.38
    
    numsopt = np.zeros((3,31))
    densopt = np.zeros((3,31))
    G = 10 **(filtergainsPrediction/20)
    Gw_db = c * filtergainsPrediction
    Gw = 10 **(Gw_db/20)
    
    for k in range(31):
        [num,den] = pareq(G[k],Gw[k],wg[k],bw[k])
        numsopt[:,k] = num
        densopt[:,k] = den

    N_freq = 2 **12
    w = np.logspace(np.log10(9),np.log10(22050), N_freq)
    H_opt = np.ones((N_freq,31), dtype=complex)
    H_opt_tot = np.ones((N_freq,1), dtype=complex)
    
    for k in range(31):
        w, h = signal.freqz(numsopt[:,k], densopt[:,k],worN=w,fs=fs)
        H_opt[:,k]= h
        H_opt_tot = H_opt[:,[k]]  * H_opt_tot
    
    fig = plt.figure(3)
    plt.semilogx(w,20*np.log10(np.abs(H_opt_tot)))
    #plt.semilogx(w,20*np.log10(np.abs(H_opt)))
    plt.plot(fc2,G_db2, "ro", markersize=5, markerfacecolor="none")
    plt.plot(fc1,filtergainsPrediction, "ro", markersize=5, markerfacecolor="none", marker="x", markeredgecolor="r")
    plt.ylabel("Pegel in dB")
    plt.xlabel("Frequenz in Hz")
    #plt.title("Frequency Response Predicted")
    plt.xticks([10, 30, 100, 1000, 3000, 10000])
    plt.yticks(np.arange(-15,15,5))
    plt.grid(which="both", linestyle="--", color="grey")
    filtergain = mlines.Line2D([], [], linestyle='None',
                          markersize=8, markerfacecolor="none", marker="x", markeredgecolor="r", label="Filter Gains")
    targetgain = mlines.Line2D([], [], linestyle='None',
                          markersize=8, markerfacecolor="none", marker="o", markeredgecolor="r", label="Target Gains")
    plt.legend(handles=[filtergain,targetgain])
    #plt.show()
    
    return fig
    

In [28]:
a = np.array([12,5,6,9,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,4,6,2]).reshape(31,1)
b = 12* np.ones(31).reshape((31,1))    
c = np.array([12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12]).reshape(31,1)    
d = 6 *np.ones(31).reshape((31,1))   

In [29]:
def thirdOctaveGEQ(commandgains):
    
    G_db = commandgains.reshape((31,1))
    start = time.time()*1000
    #for i in range(1):
    [numsopt,densopt,fs,fc2,G_db2,G2opt_db,fc1,bw] = initGEQ(G_db) #reshape to make it 31x1 instead of 1x31  
    
    #end = time.time()*1000
    #print(end - start)

    fig = plot(numsopt,densopt,fs,fc2,G_db2,G2opt_db,fc1)
    plt.show()
    #plotAbsoluteError(numsopt,densopt,fs,G_db,fc1)
    plotGdbError(G_db,fs,fc2,fc1,bw)
    #[MSE, RMSE, MAE] = ErrorFunction(numsopt,densopt,fs,G_db,fc1)
    
    return 

In [30]:
thirdOctaveGEQ(a)

C:/Users/Robert/AppData/Local/Temp/xpython_8596/1534437943.py:52: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_8596/4126492185.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_8596/4126492185.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with 

In [31]:
def thirdOctaveGEQwithPredictions(commandGains,filterGainsPrediction):
    
    G_db = commandGains
    FilterGains = filterGainsPrediction
    
    [numsopt,densopt,fs,fc2,G_db2,G2opt_db,fc1,bw] = initGEQ(G_db.reshape(31,1))
    fig = plot(numsopt,densopt,fs,fc2,G_db2,G2opt_db,fc1)
    fig_predict = plotPredictions(FilterGains,G_db,fs,fc2,fc1,bw)
    plt.show()
    
    return

In [32]:
def createOutputData():
    
    dataInput = loadtxt('dataInputLarge2.csv', delimiter=',')
    dataOutput = np.zeros((len(dataInput),31))
    dataError = np.zeros((len(dataInput),3))

    for i in range((len(dataInput))):
        [numsopt,densopt,fs,fc2,G_db2,G2opt_db,fc1,bw] = initGEQ(dataInput[i].reshape(31,1))
        dataOutput[i] = G2opt_db.T
        dataError[i] = ErrorFunction(numsopt,densopt,fs,dataInput[i].reshape(31,1),fc1)
    
    # define data
    data = asarray(dataOutput, dtype=np.float64,)
    # save to csv file
    savetxt('dataOutputLarge2.csv', data, delimiter=',')
    
    # define data
    dataErr = asarray(dataError, dtype=np.float64,)
    # save to csv file
    savetxt('dataErrorLarge2.csv', dataErr, delimiter=',')
    
    

In [33]:
#createOutputData()

In [34]:
def Errors():
    
    dataError = loadtxt("dataErrorLarge2.csv",delimiter=',').T
  

    maxMSE = np.max(dataError[0])
    maxRMSE = np.max(dataError[1])
    maxMAE = np.max(dataError[2])
   
    minMSE = np.min(dataError[0])
    minRMSE = np.min(dataError[1])
    minMAE = np.min(dataError[2])
    
    return maxMSE,maxRMSE,maxMAE, minMSE, minRMSE, minMAE

In [35]:
Errors()

(0.1593602249890968,
 0.3991994801964261,
 0.3367749357958159,
 0.008910439860896917,
 0.09439512625605687,
 0.07756281079307444)